In [24]:
from dados_de_mercado import Dados_de_mercado
import pandas as pd
from datetime import datetime, timedelta
import time
import os
import matplotlib.pyplot as plt

from fintz import Fintz

In [25]:
os.chdir("C:\\Users\\eduar\\dev\\github\\dados_fundamentalistas_fintz\\atualizado1q24")

In [26]:
precos = pd.read_parquet('historico_acoes.parquet')
ebit = pd.read_parquet('EBIT.parquet')
receita_liquida = pd.read_parquet('ReceitaLiquida.parquet')
custos = pd.read_parquet('Custos.parquet')
lucro_liquido = pd.read_parquet('LucroLiquido.parquet')
ativo_circulante = pd.read_parquet('AtivoCirculante.parquet')
ativo_nao_circulante = pd.read_parquet('AtivoNaoCirculante.parquet')
ativo_total = pd.read_parquet('AtivoTotal.parquet')
caixa = pd.read_parquet('CaixaEquivalentes.parquet')
divida_bruta = pd.read_parquet('DividaBruta.parquet')
divida_liquida = pd.read_parquet('DividaLiquida.parquet')
ebitda = pd.read_parquet('EBITDA.parquet')
passivo_circulante = pd.read_parquet('PassivoCirculante.parquet')
passivo_nao_circulante = pd.read_parquet('PassivoNaoCirculante.parquet')
passivo_total = pd.read_parquet('PassivoTotal.parquet')
patrimonio_liquido = pd.read_parquet('PatrimonioLiquido.parquet')
roe = pd.read_parquet('ROE.parquet')
p_l = pd.read_parquet('P_L.parquet')
p_vp = pd.read_parquet('P_VP.parquet')
dy = pd.read_parquet('DividendYield.parquet')
roic = pd.read_parquet('ROIC.parquet')
vpa = pd.read_parquet('VPA.parquet')
lpa = pd.read_parquet('LPA.parquet')
divida_bruta_patrimonio_liquido = pd.read_parquet('DividaBruta_PatrimonioLiquido.parquet')
divida_liquida_patrimonio_liquido = pd.read_parquet('DividaLiquida_PatrimonioLiquido.parquet')
divida_liquida_ebit = pd.read_parquet('DividaLiquida_EBIT.parquet')
divida_liquida_ebitda = pd.read_parquet('DividaLiquida_EBITDA.parquet')
valor_de_mercado = pd.read_parquet('ValorDeMercado.parquet')
ev = pd.read_parquet('EV.parquet')

In [27]:
os.chdir("C:\\Users\\eduar\\dev\\github\\radar_fundamentalista")

In [28]:
tickers = ['RRRP3', 'ABEV3','AZUL4','B3SA3','BBSE3','BBDC3','BBDC4','BBAS3','BRKM5','BRFS3','BPAC11','CMIG4','CIEL3','COGN3',
 'CPLE6','CSAN3','CMIN3','CVCB3','CYRE3','ELET3','EMBR3','EQTL3','GGBR4','GOAU4','NTCO3','HAPV3','IRBR3','ITSA4','ITUB4','JBSS3',
 'KLBN11','RENT3','LREN3','LWSA3','MGLU3','MRFG3','BEEF3','MRVE3','PCAR3','PETR4','PRIO3','PETZ3','SBSP3','SANB11','CSNA3','SLCE3',
 'SUZB3','TAEE11','VIVT3','TIMS3','UGPA3','USIM5','VALE3','WEGE3','YDUQ3']

In [29]:
# Tickers que estão no radar e não estão entre as principais ações da bolsa:
tickers_radar = ['CMIG3','TAEE3','TAEE4','CPLE3','CSMG3','SAPR11','SAPR3','SAPR4','BRSR3','BRSR5','BRSR6','ITSA3','SANB3','SANB4','ITUB3',
'PSSA3','WIZC3','ENAT3']

In [30]:
for ticker in tickers_radar:
    tickers.append(ticker)

In [31]:
anos = list(range(14,25,1))
anos

[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [32]:
df_list = []

contador = 0
for ticker in tickers:

    # Caixa
    try:
        dados_caixa = caixa[caixa['ticker']==ticker]
        dados_caixa_list = []
        for ano in anos:
            dado = dados_caixa[dados_caixa['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_caixa_list.append(dado)

        ultimo_caixa = dados_caixa_list[-1]
        caixa_anterior = ultimo_caixa
        score = 0
        for i in range(1,6):
            caixa_ano = dados_caixa_list[-(i+1)]

            if ultimo_caixa > caixa_ano:
                score += 0.5

            if caixa_anterior > caixa_ano:
                score += 0.5

            if ultimo_caixa < caixa_ano*0.5 or caixa_anterior < caixa_ano*0.67:
                score -= 1

            caixa_anterior = caixa_ano

        score /= 5
        score_caixa = score #ok
    except:
        score_caixa = 0

    # evolução divida longo prazo
    try:
        dados_divida_longa = passivo_nao_circulante[passivo_nao_circulante['ticker']==ticker]
        dados_divida_longa_list = []
        for ano in anos:
            dado = dados_divida_longa[dados_divida_longa['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_divida_longa_list.append(dado)

        ultima_divida = dados_divida_longa_list[-1]
        ultima_divida = round(ultima_divida,2)
        divida_anterior = ultima_divida
        score = 0
        for i in range(1,6):
            divida_ano = dados_divida_longa_list[-(i+1)]
            divida_ano = round(divida_ano,2)

            if ultima_divida < divida_ano:
                score += 0.5

            if divida_anterior < divida_ano:
                score += 0.5

            if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
                score -= 1

            divida_anterior = divida_ano


        score /= 5
        score_divida_longo_prazo = score #ok
    except:
        score_divida_longo_prazo = 0

    # Dados número de ações:
    try:
        dados_patrimonio_liquido = patrimonio_liquido[patrimonio_liquido['ticker']==ticker]
        dados_vpa = vpa[vpa['ticker']==ticker]

        numero_acoes_list = []
        for ano in anos:
            dado_pl = dados_patrimonio_liquido[dados_patrimonio_liquido['data']==f'20{ano}-01-01']['valor'].values[0]
            dado_vpa = dados_vpa[dados_vpa['data']==f'20{ano}-01-01']['valor'].values[0]
            num_acoes = dado_pl/dado_vpa
            num_acoes = num_acoes/1000000
            num_acoes = round(num_acoes,-1)
            numero_acoes_list.append(num_acoes)

        ultimo_num = numero_acoes_list[-1]
        score = 0
        ano = 2024
        for i in range(1,6):
            num_ano = numero_acoes_list[-(i+1)]
            if ultimo_num < num_ano*0.99:
                score += 1
            elif ultimo_num > num_ano*1.01:
                score -= 1

            ultimo_num = num_ano
            ano -= 1

        score /= 5
        score_num_acoes = score #ok
    except:
        score_num_acoes = 0

    # Dados dívida bruta/patrimonio liqdo
    try:
        dados_divida_pl = divida_bruta_patrimonio_liquido[divida_bruta_patrimonio_liquido['ticker']==ticker]

        dados_divida_pl_list = []
        for ano in anos:
            dado = dados_divida_pl[dados_divida_pl['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_divida_pl_list.append(dado)

        ultima_divida = dados_divida_pl_list[-1]
        ultima_divida = round(ultima_divida,2)
        divida_anterior = ultima_divida
        score = 0
        for i in range(1,6):
            divida_ano = dados_divida_pl_list[-(i+1)]
            divida_ano = round(divida_ano,2)

            if ultima_divida < divida_ano:
                score += 0.5

            if divida_anterior < divida_ano:
                score += 0.5

            if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
                score -= 1

            divida_anterior = divida_ano

        score /= 5
        score_divida_pl = score #ok
    except:
        score_divida_pl = 0

    # Dados Lucro liquido e custos
    try:
        dados_lucro_liquido = lucro_liquido[lucro_liquido['ticker']==ticker]
        dados_custos = custos[custos['ticker']==ticker]

        dados_lucro_list = []

        for ano in anos:
            dado = dados_lucro_liquido[dados_lucro_liquido['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_lucro_list.append(dado)

        ultimo_lucro = dados_lucro_list[-1]
        lucro_anterior = ultimo_lucro
        score = 0
        for i in range(1,6):
            lucro_ano = dados_lucro_list[-(i+1)]

            if ultimo_lucro > lucro_ano:
                score += 0.5

            if lucro_anterior > lucro_ano:
                score += 0.5

            if ultimo_lucro < lucro_ano*0.67 or lucro_anterior < lucro_ano*0.67:
                score -= 1

            lucro_anterior = lucro_ano

        score /= 5
        score_lucro = score #ok

        dados_custos_list = []

        for ano in anos:
            dado = dados_custos[dados_custos['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_custos_list.append(dado)

        ultimo_custo = dados_custos_list[-1]
        custo_anterior = 1
        score = 0
        for i in range(1,6):
            custo_ano = dados_custos_list[-(i+1)]

            if ultimo_custo > custo_ano:
                score += 0.5

            if custo_anterior > custo_ano:
                score += 0.5

            if ultimo_custo < custo_ano*1.5 or custo_anterior < custo_ano*1.5:
                score -= 1

            custo_anterior = custo_ano

        score /= 5
        score_custos = score #ok
    except:
        score_lucro = 0
        score_custos = 0

    # Dados dividendo
    try:
        dados_preco = precos[precos['ticker']==ticker]
        dados_dy = dy[dy['ticker']==ticker]
        dados_lpa = lpa[lpa['ticker']==ticker]

        dividendos_list = []
        payout_list = []
        for ano in anos:

            data_inicio_ano = pd.to_datetime(f'20{ano}-01-01')
            primeira_data_apos_inicio_ano = dados_preco.loc[dados_preco['data'] > data_inicio_ano, 'data'].min()
            dado_preco = dados_preco.loc[dados_preco['data'] == primeira_data_apos_inicio_ano, 'preco_abertura'].values[0]
            dado_dy = dados_dy.loc[dados_dy['data'] == primeira_data_apos_inicio_ano, 'valor'].values[0]
            dado_lpa = dados_lpa.loc[dados_lpa['data'] == primeira_data_apos_inicio_ano, 'valor'].values[0]

            dividendos = dado_preco*dado_dy
            dividendos_list.append(dividendos)

            payout = dividendos/dado_lpa
            payout_list.append(payout)
            
        ultimo_div = dividendos_list[-1]
        div_anterior = 99999990
        score = 0
        for i in range(1,6):
            div_ano = dividendos_list[-(i+1)]
            payout_ano = payout_list[-(i+1)]

            if ultimo_div > div_ano:
                score += 0.5

            if div_anterior > div_ano:
                score += 0.5

            if ultimo_div < div_ano*0.5 or div_anterior < div_ano*0.5:
                score -= 1

            # if payout_ano > 0.8:
            #     score += 0.5
            # elif payout_ano > 0.6:
            #     score += 0.3
            # elif payout_ano > 0.4:
            #     score += 0.1

            if payout_ano > 0:
                score *= payout_ano
            else:
                score *= 0.1

            div_anterior = div_ano

        score /= 5
        score = round(score,4)
        score_div = score #
    except:
        score_div = 0

    # Divida liqu / pl
    try:
        dados_div_liq_pl = divida_liquida_patrimonio_liquido[divida_liquida_patrimonio_liquido['ticker']==ticker]

        dados_divida_liq_pl_list = []
        for ano in anos:
            dado = dados_div_liq_pl[dados_div_liq_pl['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_divida_liq_pl_list.append(dado)

        ultima_divida = dados_divida_liq_pl_list[-1]
        ultima_divida = round(ultima_divida,2)
        divida_anterior = 100
        score = 0
        for i in range(1,6):
            divida_ano = dados_divida_liq_pl_list[-(i+1)]
            divida_ano = round(divida_ano,2)

            if ultima_divida < divida_ano:
                score += 0.5

            if divida_anterior < divida_ano:
                score += 0.5

            if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
                score -= 1

            divida_anterior = divida_ano

        score /= 5
        score_div_liq_pl = score #ok
    except:
        score_div_liq_pl = 0

    # Divida liqu / ebitda
    try:
        dados_div_ebitda = divida_liquida_ebitda[divida_liquida_ebitda['ticker']==ticker]

        dados_div_ebitda_list = []
        for ano in anos:
            dado = dados_div_ebitda[dados_div_ebitda['data']==f'20{ano}-01-01']['valor'].values[0]
            dados_div_ebitda_list.append(dado)

        ultima_divida = dados_div_ebitda_list[-1]
        ultima_divida = round(ultima_divida,2)
        divida_anterior = 100
        score = 0
        for i in range(1,6):
            divida_ano = dados_div_ebitda_list[-(i+1)]
            divida_ano = round(divida_ano,2)

            if ultima_divida < divida_ano:
                score += 0.5

            if divida_anterior < divida_ano:
                score += 0.5

            if ultima_divida > divida_ano*2 or divida_anterior > divida_ano*2:
                score -= 1

            divida_anterior = divida_ano

        score /= 5
        score_div_ebitda = score
    except:
        score_div_ebitda = 0

    # Formando o dataframe parcial:
    dic = {
        'ticker': ticker,
        'caixa': score_caixa,
        'divida_longo_prazo': score_divida_longo_prazo,
        'num_acoes': score_num_acoes,
        'divida_pl': score_divida_pl,
        'lucro': score_lucro,
        'custos': score_custos,
        'dividendos': score_div,
        'divida_liq_pl': score_div_liq_pl,
        'divida_ebitda': score_div_ebitda
    }

    contador += 1
    print(f'{round(contador/len(tickers),2)*100}% decorrido')
    temp_df = pd.DataFrame(dic, index=[0])
    df_list.append(temp_df)

1.0% decorrido
3.0% decorrido
4.0% decorrido
5.0% decorrido
7.000000000000001% decorrido
8.0% decorrido
10.0% decorrido
11.0% decorrido
12.0% decorrido
14.000000000000002% decorrido
15.0% decorrido
16.0% decorrido
18.0% decorrido
19.0% decorrido
21.0% decorrido
22.0% decorrido
23.0% decorrido
25.0% decorrido
26.0% decorrido
27.0% decorrido
28.999999999999996% decorrido
30.0% decorrido
32.0% decorrido
33.0% decorrido
34.0% decorrido
36.0% decorrido
37.0% decorrido
38.0% decorrido
40.0% decorrido
41.0% decorrido
42.0% decorrido
44.0% decorrido
45.0% decorrido
47.0% decorrido
48.0% decorrido
49.0% decorrido
51.0% decorrido
52.0% decorrido
53.0% decorrido
55.00000000000001% decorrido
56.00000000000001% decorrido
57.99999999999999% decorrido
59.0% decorrido
60.0% decorrido
62.0% decorrido
63.0% decorrido
64.0% decorrido
66.0% decorrido
67.0% decorrido
68.0% decorrido
70.0% decorrido
71.0% decorrido
73.0% decorrido
74.0% decorrido
75.0% decorrido
77.0% decorrido
78.0% decorrido
79.0% decorri

In [33]:
score = pd.concat(df_list)

In [34]:
colunas = ['caixa', 'divida_longo_prazo', 'num_acoes', 'divida_pl',
       'lucro', 'custos', 'dividendos', 'divida_liq_pl', 'divida_ebitda']

score_list = []
for i in range(len(score)):
    linha = score.iloc[i]
    score_total = 0
    for coluna in colunas:
        score_total += linha[coluna]
    score_list.append(score_total)

score['total'] = score_list
score = score.sort_values('total', ascending = False)
score = score[score['total']!=0]

In [36]:
score.describe()

,caixa,divida_longo_prazo,num_acoes,divida_pl,lucro,custos,dividendos,divida_liq_pl,divida_ebitda,total
count,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,0.450000,0.076562,-0.178125,0.27500,0.200000,-0.123437,-0.013116,0.004687,-0.043750,0.647822
std,0.381309,0.397109,0.304708,0.36947,0.519768,0.375750,0.191876,0.339668,0.348637,1.581586
min,-0.700000,-0.800000,-0.800000,-0.70000,-0.900000,-0.900000,-1.118700,-0.900000,-1.000000,-4.918700
25%,0.300000,-0.100000,-0.400000,0.00000,-0.100000,-0.500000,-0.087975,-0.100000,-0.200000,-0.198075
50%,0.500000,0.000000,0.000000,0.20000,0.200000,-0.100000,0.000000,0.000000,0.000000,0.591900
75%,0.800000,0.300000,0.000000,0.52500,0.600000,0.100000,0.065875,0.200000,0.125000,1.739825
max,1.000000,0.800000,0.600000,0.90000,1.000000,0.800000,0.294500,0.600000,0.600000,3.599700


In [35]:
score

,ticker,caixa,divida_longo_prazo,num_acoes,divida_pl,lucro,custos,dividendos,divida_liq_pl,divida_ebitda,total
0,UGPA3,0.8,0.6,-0.2,0.8,0.5,0.3,-0.1003,0.4,0.5,3.5997
0,CMIG4,0.4,0.7,-0.6,0.9,0.4,0.3,0.1523,0.6,0.6,3.4523
0,GOAU4,0.2,0.7,0.0,0.9,-0.1,0.2,0.2572,0.6,0.6,3.3572
0,CMIG3,0.4,0.7,-0.6,0.9,0.4,0.3,0.0275,0.6,0.6,3.3275
0,GGBR4,0.2,0.7,0.0,0.9,-0.1,0.2,0.2239,0.6,0.5,3.2239
...,...,...,...,...,...,...,...,...,...,...,...
0,PRIO3,0.6,-0.8,-0.6,0.3,0.4,-0.9,0.0000,-0.4,-0.2,-1.6000
0,BRKM5,0.8,-0.2,0.0,-0.5,-0.9,0.2,-0.2121,-0.3,-0.6,-1.7121
0,MGLU3,0.4,-0.3,-0.6,-0.6,0.5,0.1,-0.0857,-0.8,-0.8,-2.1857
0,MRVE3,0.1,0.0,-0.4,0.1,-0.9,-0.1,-0.1847,-0.3,-0.8,-2.4847
